In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import os

In [2]:
# GPU SHIT
physical_devices = tf.config.experimental.list_physical_devices()
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
with open(os.path.join('data','drake_txt.txt'),'r',encoding="utf8") as f:
    raw_text = f.read()

raw_text[:100]

'\nPut my feelings on ice\nAlways been a gem\nCertified lover boy, somehow still heartless\nHeart is only'

In [14]:
# Process text and remove most punctuations 
processed_text = raw_text.lower()
processed_text = re.sub(r'[^\x00-\x7f]',r'',processed_text)
processed_text = re.sub(r'[.,\/#!$%\^&\*?;:{}=\-_`~()"]',r'',processed_text)

In [22]:
# process words and map word indexes

# this little bit is to count line breaks as words
lines = processed_text.split("\n")
all_words = []
for line in lines:
    for word in line.split():
        all_words.append(word)
    all_words.append('\n')

words = sorted(list(set(all_words)))
word_indices = dict((c,i) for i, c in enumerate(words))
indices_word = dict((i,c) for i, c in enumerate(words))

In [23]:
# make sentences
maxlen = 20
step = 3
sentences = []
next_words = []
for i in range(0, len(all_words) - maxlen, step):
    sentences.append(all_words[i: i + maxlen])
    next_words.append(all_words[i + maxlen])
len(sentences)
sentences[0]

['\n',
 'put',
 'my',
 'feelings',
 'on',
 'ice',
 '\n',
 'always',
 'been',
 'a',
 'gem',
 '\n',
 'certified',
 'lover',
 'boy',
 'somehow',
 'still',
 'heartless',
 '\n',
 'heart']

In [24]:
# Vectorization
x = np.zeros((len(sentences),maxlen,len(words)),dtype=np.bool)
y = np.zeros((len(sentences),len(words)),dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1

In [25]:
x.shape

(55940, 20, 9095)

In [26]:
y.shape

(55940, 9095)

In [28]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(words))))
model.add(Dense(len(words),activation='softmax'))

In [29]:
model.compile(optimizer=RMSprop(lr=0.01), loss='categorical_crossentropy')

In [54]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds / temperature)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    start_index = random.randint(0, len(all_words) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("---------- temperature:", temperature)

        generated = ''
        sentence = all_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)
        print("---------- Generated with seed: '" + ' '.join(sentence) + "'")
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1,maxlen,len(words)))
            for t, word in enumerate(sentence):
                x_pred[0,t,word_indices[word]] = 1
            
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds,temperature)
            next_word = indices_word[next_index]  + ' '

            generated += next_word
            sentence = sentence[1:] + [next_word.strip(' ')]

            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()

In [55]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [56]:
model.fit(x,y,batch_size=128,epochs=5,callbacks=[print_callback])

Epoch 1/5
438/438 [==============================] - 11s 25ms/step - loss: 3.0568
---------- temperature: 0.2
---------- Generated with seed: 'why you wanna be my love my love my love 
 why you wanna be my love my love my'
why you wanna be my love my love my love 
 why you wanna be my love my love mylove my love 
 why you want to why you want to what you want to what you ask i have more i gone go somebody million tell that 
 i do too much trying to tell you started no 
 need to be over were i fuck up 
 to call damn i got try to get my face 
 they always she a you to call me home who they look like to do with if they working 
 money and say ain't never far came me i never give shit 
 i year so give me a bad i've been make me only probably 
 something feelin' on next wrong ayy 
 same nigga that white never on would enough for try to come to do off as you in the high in this bitch i came 
 it's like they say you check anything i'm everybody here 
 take this shit 
 bad street gotta hey hey

 baby like want to can't on the can't real 
 cause i done the money go town 
 friend take my days all from why 
 no fuckin' look it's real like 
 
 low man i'm fuckin' way 
 they little said you should did you lie 
 i came and before but just you need to im 
 i don't think i really take all out the like a man ask used to be back when when we we had to dont bag young 
 i was change but you some only on play just on think them on nice fit doin' lean it's should low the wrong 
 look should we take a than just a new new new new me 
 aw yeah yeah yeah standin' 40 are you 
 left for on you never you deserve love 
 come through girl you where i'll be at 
 
 don't fuck up right i come with how i wayne 
 left you right here and you cost shirt fittin' mile should city where you could an uh what you you you and now 
 you never see before too friends are can help for him see i'm left through the the meal doin' niggas make you too fuck all you just gotta gotta take all think 
 make you too much to 

 you get right off back 
 not like my work or i hate it 
 if you get it then out around y'all start won't it hit it 
 away like i'm kick that still doin' 'em 
 and long them but they be from me to still last for any for always 
 like gettin' went to only got the same please gettin' throw my got like money was words 
 they used to room 
 but it's do not to what with fact 
 but runnin' it's was trying to 
 oh nigga from my weekend then 
 cause i told her said this is didn't 
 
 
 talk should we ya make niggas talkin' like wish it's like i can't make it 
 might you niggas drizzy shawty while she wanna see them truth 
 
 
 look 
 
 
 got a lot of money in your everything nigga 
 an tell you 
 you don't care 
 about you down girl 
 you make me feel like you look get sick 
 lot of you takin' take man 
 hear what these then low you 
 wanna be up and man that's guess that's me me 
 women tell rest me when you right to miami 
 
 
 
 in the club with the streets man 
 all my been make you'll sai

 girl too that pull you my take season sure you you need where to give me me man you change but i'm with no one then checks for can in your morning that's one heat for over phone made always can't it right no 
 
 
 i need a picture watch road a place 
 girl we know is it's so blind probably so it's yours 
 these uh say you play you and say to you you 
 
 
 ayy 
 god my though whole ho 
 whole just to learn 'em a way 
 out over still over on the they they had nigga 
 should be here baby hair over 
 yeah night 
 
 team never shit 
 
 
 you used to 
 me let my go than uh 'em up hannenin' i'll truth 
 i you be worst man i just give you 
 it's just been getting solo and bands and all you the things do 
 man you know set the looking down 
 'cause i was gon' be the time hold the up 
 baby you just are and 
 my pop tell maybe these days 
 they got a lot to someone that you know me 
 through the last damn that the been 
 ice hell a didn't with your mean ayy 
 i find in the top just your nigga 


 play he niggas gettin' can't those picture in five always 
 now you can think so are your your phone now 
 that you used to never loved free 
 said your had this these let's do all enough 
 when i love the hard way on left make come friends me 
 was might only ya ya got here for after alone 
 after your looking high around no as shit i'm in the soul 
 i been just ass night like a want years record 
 'cause more always because a girl what i love 
 and you like friends when i could've lose and i need since when work i'd 
 damn if you only feel what i dont 
 inside just oh yeah yeah your drake 'cause 
 and it my 'cause uh already girl on your new lot on 
 who are you getting live see so us so old can 'cause the ones ya 
 now i'm really see and some where chick stay at all 
 i can't could never never 
 the stay kill you all friends 
 night i know i miss us to man 
 all the perfect funny the mind we us 
 i just ashamed the old money sit this shit through 
 him been niggas all them pop it's